In [ ]:

import pandas as pd
import os 
import cv2
import numpy as np
from statistics import mean
import matplotlib.pyplot as plt
import random 
from data import *


In [ ]:
df = pd.DataFrame()
   
df = data_frame(df, "D:\Lab Python\Lab_1\dataset\ rose", "rose")
df = data_frame(df, "D:\Lab Python\Lab_1\dataset\ tulip", "tulip")
print(df)

In [ ]:
df = dimension(df)

In [ ]:
stat(df)


In [ ]:
df2 = pd.DataFrame()
df2 = filter_tag(df, 5)

In [ ]:
df3 = pd.DataFrame()
df3 = filter_dimensions(df, 0, 300, 460)

In [ ]:
df4 = pd.DataFrame() 
df4 = pixel(df, 0)

In [ ]:
b, g, r = hist(df, 0)
output_hist(b, g, r)